<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning_(SSL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class ContrastiveModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ContrastiveModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.projection_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, x1, x2):
        z1 = self.encoder(x1)
        z2 = self.encoder(x2)
        p1 = self.projection_head(z1)
        p2 = self.projection_head(z2)
        return z1, z2, p1, p2

def contrastive_loss(p1, p2, z1, z2):
    z1 = z1.detach()
    z2 = z2.detach()
    p1 = F.normalize(p1, dim=1)
    p2 = F.normalize(p2, dim=1)
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)
    loss = (2 - F.cosine_similarity(p1, z2, dim=1).sum() - F.cosine_similarity(p2, z1, dim=1).sum()).mean()
    return loss

input_dim = 128
hidden_dim = 256
model = ContrastiveModel(input_dim, hidden_dim)  # Remove .to('cuda')
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Dummy data for training
x1 = torch.rand(64, input_dim)  # Remove .to('cuda')
x2 = torch.rand(64, input_dim)  # Remove .to('cuda')

for epoch in range(100):
    optimizer.zero_grad()
    z1, z2, p1, p2 = model(x1, x2)
    loss = contrastive_loss(p1, p2, z1, z2)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")